# Tool Calling with Agents and using Threads

In this sample we will be creating agents and use a tool (function) to provide answers to questions. We will also be creating a thread and then doing a follow-up with the agent to ask for more information.

You should have the following two environment variables already created and initialized:

* `AZURE_AI_PROJECT_ENDPOINT`: Your Azure AI project endpoint
* `AZURE_AI_MODEL_DEPLOYMENT_NAME`: The name of your model deployment

In [ ]:
import asyncio
import os
from random import randint
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

project_endpoint = os.environ.get('AZURE_AI_PROJECT_ENDPOINT')
model_name = os.environ.get('AZURE_AI_MODEL_DEPLOYMENT_NAME')

print(f"Project endpoint: {project_endpoint}")
print(f"Deployment name: {model_name}")

In [ ]:
# 🎲 Tool Function: Random Destination Generator
# This function will be available to the agent as a tool
# The agent can call this function to get random vacation destinations
def get_random_destination() -> str:
    """Get a random vacation destination.
    
    Returns:
        str: A randomly selected destination from our predefined list
    """
    # List of popular vacation destinations around the world
    destinations = [
        "Rome", "Italy",
        "Barcelona, Spain",
        "Paris, France", 
        "Berlin, Germany",
        "Tokyo, Japan",
        "Sydney, Australia",
        "New York, USA",
        "Cairo, Egypt",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bali, Indonesia"
    ]
    # Return a random destination from the list
    return destinations[randint(0, len(destinations) - 1)]

In [ ]:
async def main():
    credential = AzureCliCredential()
    chat_client = AzureAIAgentClient(async_credential=credential)
    agent = ChatAgent(chat_client, name="Vacation Planner",
                instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations.",
                tools=[get_random_destination]
            )
                                                             
    thread = agent.get_new_thread()
    result = await agent.run("Please plan a trip for me", thread=thread)
    print(result.text)

    second_result = await agent.run("I do not like that destination. Please plan another vacation for me", thread=thread)
    print(second_result.text)

    await chat_client.close()
    

In [ ]:
await main()